In [1]:
import pandas as pd

In [2]:
def run_coint_test(etf, tickers, start, end):
    coint_data = pd.DataFrame(columns=['ticker', 't-stat', 'pval'])
    etf_data = yf.download(etf, start = start, end = end)
    etf_data = etf_data[['Close']]
    etfLogPrice = np.log(etf_data['Close'].values)
    for i, t in enumerate(tickers):
        t_data = yf.download(t, start = start, end = end)
        t_data = t_data[['Close']]
        tLogPrice = np.log(t_data['Close'].values)
        if t_data.isnull().values.any() or len(etfLogPrice) != len(tLogPrice):
            continue
        t_stat, pval, crit_values = coint(etfLogPrice, tLogPrice)
        coint_data.loc[i] = [t, t_stat, pval]
        print(t)
    return coint_data.sort_values(by='t-stat', ascending=True)

In [3]:
# Need a way to download only the NASDAQ100 tickers available in the year we are running the cointegration test on and the year immediately after
tickers = pd.read_csv("./data/NASDAQ100Components")
tickers = tickers['Ticker'].tolist()
etf = 'QQQ'
start = '2018-11-01'
end = '2019-10-31'
cointegration_results = run_coint_test(etf, tickers, start, end)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ATVI
[*********************100%***********************]  1 of 1 completed
ADBE
[*********************100%***********************]  1 of 1 completed
AMD
[*********************100%***********************]  1 of 1 completed
ALXN
[*********************100%***********************]  1 of 1 completed
ALGN
[*********************100%***********************]  1 of 1 completed
GOOGL
[*********************100%***********************]  1 of 1 completed
GOOG
[*********************100%***********************]  1 of 1 completed
AMZN
[*********************100%***********************]  1 of 1 completed
AMGN
[*********************100%***********************]  1 of 1 completed
ADI
[*********************100%***********************]  1 of 1 completed
ANSS
[*********************100%***********************]  1 of 1 completed
AAPL
[*********************100%******************

In [4]:
cointegration_results.head(15)

,ticker,t-stat,pval
53,JD,-3.801199,0.013506
73,NXPI,-3.702010,0.018217
10,ANSS,-3.680510,0.019410
60,LULU,-3.612945,0.023612
88,SNPS,-3.585750,0.025514
11,AAPL,-3.529315,0.029884
89,TMUS,-3.490866,0.033214
2,AMD,-3.457486,0.036355
31,CPRT,-3.367093,0.046139
100,XEL,-3.316501,0.052508


In [5]:
len(cointegration_results)

97

In [6]:
def run_coint_test_subset(etf, tickers, start, end):
    coint_data = pd.DataFrame(columns=['ticker', 't-stat', 'pval'])
    etf_data = yf.download(etf, start = start, end = end)
    etf_data = etf_data[['Close']]
    etfLogPrice = np.log(etf_data['Close'].values)
    for i, t_list in enumerate(tickers):
        if (((i / len(tickers)) * 100) % 10 == 0):
            print(((i / len(tickers)) * 100), '% done')
        t_data = yf.download(t_list, start = start, end = end)
        t_data = t_data[['Close']]
        t_data['agg'] = sum([t_data['Close'][t] for t in t_list])
        tLogPrice = np.log(t_data['agg'].values)
        if t_data.isnull().values.any() or len(etfLogPrice) != len(tLogPrice):
            continue
        t_stat, pval, crit_values = coint(etfLogPrice, tLogPrice)
        coint_data.loc[i] = [t_list, t_stat, pval]
    return coint_data.sort_values(by='t-stat', ascending=True)

In [7]:
subset = cointegration_results.loc[cointegration_results['pval'] <= 0.1]
tickers = subset['ticker'].tolist()
tickers_subsets = []
for i in range(2, len(tickers) + 1):
    for subset in itertools.combinations(tickers, i):
        tickers_subsets += [subset]

etf = 'QQQ'
start = '2018-11-01'
end = '2019-10-31'
synthetic_asset_rank = run_coint_test_subset(etf, tickers_subsets, start, end)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  2 of 2 completed
('JD', 'NXPI')
[*********************100%***********************]  2 of 2 completed
('JD', 'ANSS')
[*********************100%***********************]  2 of 2 completed
('JD', 'LULU')
[*********************100%***********************]  2 of 2 completed
('JD', 'SNPS')
[*********************100%***********************]  2 of 2 completed
('JD', 'AAPL')
[*********************100%***********************]  2 of 2 completed
('JD', 'TMUS')
[*********************100%***********************]  2 of 2 completed
('JD', 'AMD')
[*********************100%***********************]  2 of 2 completed
('JD', 'CPRT')
[*********************100%***********************]  2 of 2 completed
('JD', 'XEL')
[*********************100%***********************]  2 of 2 completed
('JD', 'CDW')
[*********************100%***********************]  2 of 2 completed
('JD', 'ORLY')
[*********

[*********************100%***********************]  3 of 3 completed
('NXPI', 'LULU', 'CPRT')
[*********************100%***********************]  3 of 3 completed
('NXPI', 'LULU', 'XEL')
[*********************100%***********************]  3 of 3 completed
('NXPI', 'LULU', 'CDW')
[*********************100%***********************]  3 of 3 completed
('NXPI', 'LULU', 'ORLY')
[*********************100%***********************]  3 of 3 completed
('NXPI', 'LULU', 'MDLZ')
[*********************100%***********************]  3 of 3 completed
('NXPI', 'LULU', 'KHC')
[*********************100%***********************]  3 of 3 completed
('NXPI', 'SNPS', 'AAPL')
[*********************100%***********************]  3 of 3 completed
('NXPI', 'SNPS', 'TMUS')
[*********************100%***********************]  3 of 3 completed
('NXPI', 'SNPS', 'AMD')
[*********************100%***********************]  3 of 3 completed
('NXPI', 'SNPS', 'CPRT')
[*********************100%***********************]  3 of 3 compl

[*********************100%***********************]  3 of 3 completed
('ANSS', 'AMD', 'ORLY')
[*********************100%***********************]  3 of 3 completed
('ANSS', 'AMD', 'MDLZ')
[*********************100%***********************]  3 of 3 completed
('ANSS', 'AMD', 'KHC')
[*********************100%***********************]  3 of 3 completed
('ANSS', 'CPRT', 'XEL')
[*********************100%***********************]  3 of 3 completed
('ANSS', 'CPRT', 'CDW')
[*********************100%***********************]  3 of 3 completed
('ANSS', 'CPRT', 'ORLY')
[*********************100%***********************]  3 of 3 completed
('ANSS', 'CPRT', 'MDLZ')
[*********************100%***********************]  3 of 3 completed
('ANSS', 'CPRT', 'KHC')
[*********************100%***********************]  3 of 3 completed
('ANSS', 'XEL', 'CDW')
[*********************100%***********************]  3 of 3 completed
('ANSS', 'XEL', 'ORLY')
[*********************100%***********************]  3 of 3 completed
(

[*********************100%***********************]  3 of 3 completed
('SNPS', 'CPRT', 'KHC')
[*********************100%***********************]  3 of 3 completed
('SNPS', 'XEL', 'CDW')
[*********************100%***********************]  3 of 3 completed
('SNPS', 'XEL', 'ORLY')
[*********************100%***********************]  3 of 3 completed
('SNPS', 'XEL', 'MDLZ')
[*********************100%***********************]  3 of 3 completed
('SNPS', 'XEL', 'KHC')
[*********************100%***********************]  3 of 3 completed
('SNPS', 'CDW', 'ORLY')
[*********************100%***********************]  3 of 3 completed
('SNPS', 'CDW', 'MDLZ')
[*********************100%***********************]  3 of 3 completed
('SNPS', 'CDW', 'KHC')
[*********************100%***********************]  3 of 3 completed
('SNPS', 'ORLY', 'MDLZ')
[*********************100%***********************]  3 of 3 completed
('SNPS', 'ORLY', 'KHC')
[*********************100%***********************]  3 of 3 completed
('S

[*********************100%***********************]  4 of 4 completed
('JD', 'ANSS', 'SNPS', 'AAPL')
[*********************100%***********************]  4 of 4 completed
('JD', 'ANSS', 'SNPS', 'TMUS')
[*********************100%***********************]  4 of 4 completed
('JD', 'ANSS', 'SNPS', 'AMD')
[*********************100%***********************]  4 of 4 completed
('JD', 'ANSS', 'SNPS', 'CPRT')
[*********************100%***********************]  4 of 4 completed
('JD', 'ANSS', 'SNPS', 'XEL')
[*********************100%***********************]  4 of 4 completed
('JD', 'ANSS', 'SNPS', 'CDW')
[*********************100%***********************]  4 of 4 completed
('JD', 'ANSS', 'SNPS', 'ORLY')
[*********************100%***********************]  4 of 4 completed
('JD', 'ANSS', 'SNPS', 'MDLZ')
[*********************100%***********************]  4 of 4 completed
('JD', 'ANSS', 'SNPS', 'KHC')
[*********************100%***********************]  4 of 4 completed
('JD', 'ANSS', 'AAPL', 'TMUS')
[***

...

[*********************100%***********************]  11 of 11 completed
('JD', 'NXPI', 'ANSS', 'LULU', 'AAPL', 'TMUS', 'AMD', 'CPRT', 'XEL', 'ORLY', 'MDLZ')
[*********************100%***********************]  11 of 11 completed
('JD', 'NXPI', 'ANSS', 'LULU', 'AAPL', 'TMUS', 'AMD', 'CPRT', 'XEL', 'ORLY', 'KHC')
[*********************100%***********************]  11 of 11 completed
('JD', 'NXPI', 'ANSS', 'LULU', 'AAPL', 'TMUS', 'AMD', 'CPRT', 'XEL', 'MDLZ', 'KHC')
[*********************100%***********************]  11 of 11 completed
('JD', 'NXPI', 'ANSS', 'LULU', 'AAPL', 'TMUS', 'AMD', 'CPRT', 'CDW', 'ORLY', 'MDLZ')
[*********************100%***********************]  11 of 11 completed
('JD', 'NXPI', 'ANSS', 'LULU', 'AAPL', 'TMUS', 'AMD', 'CPRT', 'CDW', 'ORLY', 'KHC')
[*********************100%***********************]  11 of 11 completed
('JD', 'NXPI', 'ANSS', 'LULU', 'AAPL', 'TMUS', 'AMD', 'CPRT', 'CDW', 'MDLZ', 'KHC')
[*********************100%***********************]  11 of 11 complet

[*********************100%***********************]  11 of 11 completed
('JD', 'NXPI', 'LULU', 'SNPS', 'AAPL', 'CPRT', 'XEL', 'CDW', 'ORLY', 'MDLZ', 'KHC')
[*********************100%***********************]  11 of 11 completed
('JD', 'NXPI', 'LULU', 'SNPS', 'TMUS', 'AMD', 'CPRT', 'XEL', 'CDW', 'ORLY', 'MDLZ')
[*********************100%***********************]  11 of 11 completed
('JD', 'NXPI', 'LULU', 'SNPS', 'TMUS', 'AMD', 'CPRT', 'XEL', 'CDW', 'ORLY', 'KHC')
[*********************100%***********************]  11 of 11 completed
('JD', 'NXPI', 'LULU', 'SNPS', 'TMUS', 'AMD', 'CPRT', 'XEL', 'CDW', 'MDLZ', 'KHC')
[*********************100%***********************]  11 of 11 completed
('JD', 'NXPI', 'LULU', 'SNPS', 'TMUS', 'AMD', 'CPRT', 'XEL', 'ORLY', 'MDLZ', 'KHC')
[*********************100%***********************]  11 of 11 completed
('JD', 'NXPI', 'LULU', 'SNPS', 'TMUS', 'AMD', 'CPRT', 'CDW', 'ORLY', 'MDLZ', 'KHC')
[*********************100%***********************]  11 of 11 completed
(

[*********************100%***********************]  11 of 11 completed
('JD', 'ANSS', 'LULU', 'SNPS', 'AAPL', 'AMD', 'CPRT', 'CDW', 'ORLY', 'MDLZ', 'KHC')
[*********************100%***********************]  11 of 11 completed
('JD', 'ANSS', 'LULU', 'SNPS', 'AAPL', 'AMD', 'XEL', 'CDW', 'ORLY', 'MDLZ', 'KHC')
[*********************100%***********************]  11 of 11 completed
('JD', 'ANSS', 'LULU', 'SNPS', 'AAPL', 'CPRT', 'XEL', 'CDW', 'ORLY', 'MDLZ', 'KHC')
[*********************100%***********************]  11 of 11 completed
('JD', 'ANSS', 'LULU', 'SNPS', 'TMUS', 'AMD', 'CPRT', 'XEL', 'CDW', 'ORLY', 'MDLZ')
[*********************100%***********************]  11 of 11 completed
('JD', 'ANSS', 'LULU', 'SNPS', 'TMUS', 'AMD', 'CPRT', 'XEL', 'CDW', 'ORLY', 'KHC')
[*********************100%***********************]  11 of 11 completed
('JD', 'ANSS', 'LULU', 'SNPS', 'TMUS', 'AMD', 'CPRT', 'XEL', 'CDW', 'MDLZ', 'KHC')
[*********************100%***********************]  11 of 11 completed
('

[*********************100%***********************]  11 of 11 completed
('NXPI', 'ANSS', 'LULU', 'SNPS', 'AAPL', 'TMUS', 'AMD', 'XEL', 'CDW', 'MDLZ', 'KHC')
[*********************100%***********************]  11 of 11 completed
('NXPI', 'ANSS', 'LULU', 'SNPS', 'AAPL', 'TMUS', 'AMD', 'XEL', 'ORLY', 'MDLZ', 'KHC')
[*********************100%***********************]  11 of 11 completed
('NXPI', 'ANSS', 'LULU', 'SNPS', 'AAPL', 'TMUS', 'AMD', 'CDW', 'ORLY', 'MDLZ', 'KHC')
[*********************100%***********************]  11 of 11 completed
('NXPI', 'ANSS', 'LULU', 'SNPS', 'AAPL', 'TMUS', 'CPRT', 'XEL', 'CDW', 'ORLY', 'MDLZ')
[*********************100%***********************]  11 of 11 completed
('NXPI', 'ANSS', 'LULU', 'SNPS', 'AAPL', 'TMUS', 'CPRT', 'XEL', 'CDW', 'ORLY', 'KHC')
[*********************100%***********************]  11 of 11 completed
('NXPI', 'ANSS', 'LULU', 'SNPS', 'AAPL', 'TMUS', 'CPRT', 'XEL', 'CDW', 'MDLZ', 'KHC')
[*********************100%***********************]  11 of 

[*********************100%***********************]  12 of 12 completed
('JD', 'NXPI', 'ANSS', 'LULU', 'AAPL', 'TMUS', 'AMD', 'CPRT', 'XEL', 'CDW', 'ORLY', 'KHC')
[*********************100%***********************]  12 of 12 completed
('JD', 'NXPI', 'ANSS', 'LULU', 'AAPL', 'TMUS', 'AMD', 'CPRT', 'XEL', 'CDW', 'MDLZ', 'KHC')
[*********************100%***********************]  12 of 12 completed
('JD', 'NXPI', 'ANSS', 'LULU', 'AAPL', 'TMUS', 'AMD', 'CPRT', 'XEL', 'ORLY', 'MDLZ', 'KHC')
[*********************100%***********************]  12 of 12 completed
('JD', 'NXPI', 'ANSS', 'LULU', 'AAPL', 'TMUS', 'AMD', 'CPRT', 'CDW', 'ORLY', 'MDLZ', 'KHC')
[*********************100%***********************]  12 of 12 completed
('JD', 'NXPI', 'ANSS', 'LULU', 'AAPL', 'TMUS', 'AMD', 'XEL', 'CDW', 'ORLY', 'MDLZ', 'KHC')
[*********************100%***********************]  12 of 12 completed
('JD', 'NXPI', 'ANSS', 'LULU', 'AAPL', 'TMUS', 'CPRT', 'XEL', 'CDW', 'ORLY', 'MDLZ', 'KHC')
[*********************100

In [11]:
synthetic_asset_rank.head(15)

,ticker,t-stat,pval
368,"(SNPS, ORLY, MDLZ)",-4.686451,0.000595
1307,"(SNPS, AMD, ORLY, MDLZ)",-4.679907,0.000611
2517,"(NXPI, SNPS, AMD, ORLY, MDLZ)",-4.664184,0.000650
874,"(NXPI, SNPS, ORLY, MDLZ)",-4.641019,0.000712
1323,"(SNPS, XEL, ORLY, MDLZ)",-4.635005,0.000728
654,"(JD, SNPS, ORLY, MDLZ)",-4.630448,0.000742
1292,"(SNPS, TMUS, ORLY, MDLZ)",-4.629990,0.000743
2022,"(JD, SNPS, AMD, ORLY, MDLZ)",-4.629665,0.000744
3310,"(SNPS, AMD, XEL, ORLY, MDLZ)",-4.623080,0.000763
4526,"(JD, SNPS, AAPL, AMD, ORLY, MDLZ)",-4.618818,0.000776


In [9]:
synthetic_asset_rank.to_csv('./data/syntheticAssetsRank.csv')

In [10]:
len(synthetic_asset_rank)

16369